In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
from cellpose import io, models, train
from cellpose.io import imread
import re
import shutil

%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
from cellpose import utils, io


#选择数据集Fluo-C2DL-MSC/Fluo-N2DH-GOWT1/Fluo-N2DH-SIM+/Fluo-N2DL-HeLa
task='Fluo-N2DH-SIM+'
data_path='./dataset'
EPOCH=600

In [ ]:

train_path='./dataset/train_temp'

os.makedirs(train_path, exist_ok=True) #创建目录

imgs=[]
masks=[]
for seq in range(1,3):
    temp_img_path=os.path.join(data_path,'train',task,'0{}'.format(seq))
    temp_gt_path=os.path.join(data_path,'train',task,'0{}_GT'.format(seq))
    temp_st_path=os.path.join(data_path,'train',task,'0{}_ST'.format(seq))
    ls_img=os.listdir(temp_img_path)

    if os.path.exists(temp_st_path):
        ls_st=os.listdir(temp_st_path)
        
    ls_gt=os.listdir(temp_gt_path)
    for img_name in ls_img:
        prefix, suffix = os.path.splitext(img_name)
        idx = re.search(r'\d+$', prefix).group()
        gt_seg_path=os.path.join(temp_gt_path,'SEG','man_seg'+idx+'.tif')
        st_seg_path=os.path.join(temp_st_path,'SEG','man_seg'+idx+'.tif')

        if os.path.exists(gt_seg_path):#若有GT则采用
            masks.append(gt_seg_path)
        else:
            masks.append(st_seg_path)#否则用ST
        imgs.append(os.path.join(temp_img_path,img_name))

for i in range(len(imgs)):
    shutil.copy(imgs[i], os.path.join(train_path,'{}_img.tif'.format(i)))
    shutil.copy(masks[i], os.path.join(train_path,'{}_mask.tif'.format(i)))




In [ ]:

io.logger_setup()

images, labels,file_names = io.load_images_labels(tdir=os.path.abspath(train_path),look_one_level_down=True,mask_filter='_mask',image_filter='_img')

# # # e.g. retrain a Cellpose model
model = models.CellposeModel(gpu=True,model_type="cyto3")


train.train_seg(model.net, train_data=images, train_labels=labels,
                            channels=[0,0], normalize=True,learning_rate=0.1,
                            n_epochs=EPOCH, model_name="cyto3for"+task)

if os.path.exists(train_path):
    shutil.rmtree(train_path) #递归删除临时目录

In [ ]:
#选择数据集Fluo-C2DL-MSC/Fluo-N2DH-GOWT1/Fluo-N2DH-SIM+/Fluo-N2DL-HeLa
task='Fluo-N2DH-GOWT1'
data_path='./dataset'

model_path='./models'
model = models.CellposeModel(gpu=True, model_type='cyto3',pretrained_model=os.path.join(model_path,'cyto3for'+task))

#在challeng进行分割
challenge_path = os.path.join(data_path,'challenge',task)

for idx in range(1,3):
    seq_path=os.path.join(challenge_path,'0{}'.format(idx))
    res_path=os.path.join(challenge_path,'0{}_RES'.format(idx))
    os.makedirs(res_path, exist_ok=True)

    files_name = os.listdir(seq_path)# 使用os.listdir()获取目录中的文件和目录列表
    files=[os.path.join(seq_path,it) for it in files_name]
    imgs = [imread(f) for f in files]
    channels = [[0,0]]

    masks, _,_= model.eval(imgs, diameter=None, channels=channels)#计算掩码结果
    for mask,file in zip(masks,files_name):#保存结果
        prefix, _ = os.path.splitext(file)
        idx = re.search(r'\d+$', prefix).group()#正则匹配图像编号
        save_path=os.path.join(res_path,'mask'+idx+'.tif')
        io.imsave(save_path,mask)



In [ ]:
model_path='./models'
model = models.CellposeModel(gpu=True, model_type='cyto3',pretrained_model=os.path.join(model_path,'cyto3for'+task))

#在train上进行分割
train_path = os.path.join(data_path,'train',task)
for idx in range(1,3):
    seq_path=os.path.join(train_path,'0{}'.format(idx))
    res_path=os.path.join(train_path,'0{}_RES'.format(idx))
    os.makedirs(res_path, exist_ok=True)

    files_name = os.listdir(seq_path)# 使用os.listdir()获取目录中的文件和目录列表
    files=[os.path.join(seq_path,it) for it in files_name]
    imgs = [imread(f) for f in files]
    channels = [[0,0]]

    masks, _,_= model.eval(imgs, diameter=None, channels=channels)#计算掩码结果
    for mask,file in zip(masks,files_name):#保存结果
        prefix, _ = os.path.splitext(file)
        idx = re.search(r'\d+$', prefix).group()#正则匹配图像编号
        save_path=os.path.join(res_path,'mask'+idx+'.tif')
        io.imsave(save_path,mask)



In [ ]:

# #取其中第一张图片
# mask=masks[0]
# image=imgs[0]

# # 获取掩模中所有独立细胞的标签（忽略背景标签0）
# unique_cells = np.unique(mask)
# unique_cells = unique_cells[unique_cells != 0]  # 排除背景

# # 创建一个图像用于可视化
# fig, ax = plt.subplots()
# ax.imshow(image, cmap='gray')  # 显示灰度背景图像

# # 为每个细胞生成一个随机颜色并绘制
# for cell in unique_cells:
#     # 生成一个随机颜色
#     cell_color = np.random.rand(3,)  # 生成三个介于0和1之间的随机数作为RGB颜色
#     # 创建一个临时掩模，仅显示当前细胞
#     temp_mask = (mask == cell)
#     # 创建一个自定义颜色映射
#     cmap = ListedColormap([(0,0,0,0), (*cell_color, 0.5)])  # 使用生成的颜色，加上半透明
#     # 绘制当前细胞
#     ax.imshow(temp_mask, cmap=cmap)

# ax.axis('off')  # 隐藏坐标轴
# plt.show()